In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import tflearn
from tflearn.data_utils import to_categorical

curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Scipy not supported!


In [2]:
df_reviews = pd.read_csv('reviews.txt', header=None)
df_labels = pd.read_csv('labels.txt', header=None)


In [3]:
#--- count how often each word is occurs
from collections import Counter

words = []
for row_num, review in df_reviews.itertuples():
    for word in review.split(' '):
        words.append(word)
            
word_count = Counter(words)

In [4]:
print (len(words))
print (words[0])
print (word_count['high'])
print (len(word_count))  #--- should be 74074

7459318
bromwell
2161
74074


In [5]:
#--- get top 10000 frequest words
frequent_words = sorted(word_count, key=word_count.get, reverse=True)[:10000]

In [6]:
print(frequent_words[:60])

['', 'the', '.', 'and', 'a', 'of', 'to', 'is', 'br', 'it', 'in', 'i', 'this', 'that', 's', 'was', 'as', 'for', 'with', 'movie', 'but', 'film', 'you', 'on', 't', 'not', 'he', 'are', 'his', 'have', 'be', 'one', 'all', 'at', 'they', 'by', 'an', 'who', 'so', 'from', 'like', 'there', 'her', 'or', 'just', 'about', 'out', 'if', 'has', 'what', 'some', 'good', 'can', 'more', 'she', 'when', 'very', 'up', 'time', 'no']


In [7]:
word_idx = 1
frequent_word = frequent_words[word_idx]
count = word_count[frequent_word]
print ("word:\'{}\', count:{}".format(frequent_word, count))

word:'the', count:336713


In [8]:
word_to_idx = {}
for idx, word in enumerate(frequent_words):
    word_to_idx[word] = idx
    
def idx(word):
    try:
        return word_to_idx[word]
    except:
        return -1
        

In [9]:
idx('ben blazado')

-1

In [10]:
def text_to_vector(text):
    
    text_vector = np.zeros(len(frequent_words))
    words = text.split(' ')
    for word in words:
        try:
            idx = word_to_idx[word]
            text_vector[idx] += 1
        except:
            next
        
    return text_vector

In [11]:
test = [0, 1, 0, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]

print (text_to_vector('The tea is for a party to celebrate '
               'the movie so she has no time for a cake')[:len(test)] == test)
       


[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True]


In [12]:
len(df_reviews.index)

25000

In [13]:
shape = (df_reviews.shape[0], len(frequent_words))
review_vectors = np.zeros(shape)
for row_num, review in df_reviews.itertuples():
    review_vectors[row_num] = text_to_vector(review)

In [14]:
print (len(review_vectors))
review_vectors[:5, :20]

25000


array([[ 18.,   9.,  27.,   1.,   4.,   4.,   6.,   4.,   0.,   2.,   2.,
          5.,   0.,   4.,   1.,   0.,   2.,   0.,   0.,   0.],
       [  5.,   4.,   8.,   1.,   7.,   3.,   1.,   2.,   0.,   4.,   0.,
          0.,   0.,   1.,   2.,   0.,   0.,   1.,   3.,   0.],
       [ 78.,  24.,  12.,   4.,  17.,   5.,  20.,   2.,   8.,   8.,   2.,
          1.,   1.,   2.,   8.,   0.,   5.,   5.,   4.,   0.],
       [167.,  53.,  23.,   0.,  22.,  23.,  13.,  14.,   8.,  10.,   8.,
         12.,   9.,   4.,  11.,   2.,  11.,   5.,  11.,   0.],
       [ 19.,  10.,  11.,   4.,   6.,   2.,   2.,   5.,   0.,   1.,   2.,
          3.,   1.,   0.,   0.,   0.,   3.,   1.,   0.,   1.]])

In [52]:
labels = np.zeros(df_reviews.shape[0])
for row_num, row_data in df_labels.itertuples():
    if row_data == 'positive':
        labels[row_num] = 1
    else:
        labels[row_num] = 0
        
indices = np.arange(df_reviews.shape[0])
np.random.shuffle(indices)
test_fraction = 0.9
split_index = int(num_records * test_fraction)

train_indices = indices[:split_index]
test_indices = indices[split_index:]



In [54]:
labels[:10]

array([1., 0., 1., 0., 1., 0., 1., 0., 1., 0.])

In [34]:
df_labels.values[:10]

array([['positive'],
       ['negative'],
       ['positive'],
       ['negative'],
       ['positive'],
       ['negative'],
       ['positive'],
       ['negative'],
       ['positive'],
       ['negative']], dtype=object)

In [58]:
trainX = review_vectors[train_indices]
trainY = to_categorical(labels[train_indices], 2)

testX = review_vectors[test_indices]
testY = to_categorical(labels[test_indices], 2)

In [63]:
trainX[:10]

array([[ 9.,  0.,  2., ...,  0.,  0.,  0.],
       [72., 20., 12., ...,  0.,  0.,  0.],
       [80., 16., 10., ...,  0.,  0.,  0.],
       ...,
       [ 6.,  3.,  9., ...,  0.,  0.,  0.],
       [38., 25., 54., ...,  0.,  0.,  0.],
       [82., 36., 22., ...,  0.,  0.,  0.]])

In [64]:
trainY[:10]

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.]])

In [83]:
import tensorflow as tf
tf.reset_default_graph()
net = tflearn.input_data([None, 10000])
net = tflearn.fully_connected(net, 256, activation='LeakyReLU')
net = tflearn.fully_connected(net, 32, activation='LeakyReLU')
net = tflearn.fully_connected(net, 2, activation='softmax')

In [84]:
net = tflearn.regression(net, optimizer='sgd', learning_rate=0.1, loss='categorical_crossentropy')
model = tflearn.DNN(net)

In [85]:
model.fit (trainX, trainY, validation_set=0.1, show_metric=True, batch_size=128, n_epoch=100)

Training Step: 15899  | total loss: 0.27467 | time: 3.369s
| SGD | epoch: 100 | loss: 0.27467 - acc: 0.8731 -- iter: 20224/20250
Training Step: 15900  | total loss: 0.26941 | time: 4.392s
| SGD | epoch: 100 | loss: 0.26941 - acc: 0.8796 | val_loss: 0.35141 - val_acc: 0.8578 -- iter: 20250/20250
--


In [ ]:
--------------------------------------------

In [ ]:
from collections import Counter
c = Counter(words)

In [ ]:
c['bromwell']
    

In [ ]:
words = [word for word in [review.split() for review in reviews]]
for review in reviews:
    for word in review.split():
        words += word
        
words = [word for word in [review.split() for review in [arr_review_value[0] for arr_review_value in 

In [ ]:
i = 0
for row in df_reviews.itertuples():
    print(row)
    print("---")
    i += 1
    if i > 3:
        break

In [ ]:
from collections import Counter

r = reviews.stack().values

In [ ]:
r = reviews.values
r[1][0].split()

In [ ]:
reviews = [review for review in reviews.values]

In [ ]:
review[1]

In [ ]:
df_review_values = [df_review_values for df_review_values in df_reviews.values[0]]

In [ ]:
words = [words for review in reviews [for df_reviews in df_reviews.values[0]]
len(words)

In [ ]:
len(b)

In [ ]:
# reviews = [review for review in df_reviews.values[0]]
#reviews[0]
#for review in reviews:
#    for word in review.split()
    
words = [word for word in [review.split() for review in [df_review_value[0] for df_review_value in df_reviews.values]]]

In [ ]:
words[1]

In [ ]:
words = [word for word in [review in df_reviews.values] ]


In [ ]:
a = "the quick brown fox jumps of the lazy dog"
words = [word for word in a.split()]
print (words)

In [ ]:
words = a.split()
print (words)

In [ ]:
a